In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
sourcedir = os.path.join('..','sourcedata')
outputdir = os.path.join('..','derivatives')

In [172]:
fpath = os.path.join(sourcedir,'memrl_data.xlsx')
data = pd.read_excel(fpath)

In [109]:
fpath = os.path.join(sourcedir,'memrl_data.xlsx')
data = pd.read_excel(fpath)[[
        'SubjNum','CueLeftPic','CueRightPic','CueChosen','Outcome'
    ]]

In [173]:
data.head()

,SubjNum,AgeGroup,ExperimenterName,RunNum,Date,Time,TrialNum,DelayCond,TrialNumbyCue,CueOnLeft,...,ButtonResp,Outcome,OutcomePic,CueStart,RT,OutcomeStart,OutcomeDuration,IntertrialDuration,ChoseGoodCue?,TrialBlock
0,101,1,nl,0,10_28,0.719317,1,1,1,1,...,1.0,1.0,indoor_(45).jpg,4.033720,0.929655,6.000884,1.506218,2.779541,0.0,1
1,101,1,nl,0,10_28,0.719317,2,1,2,0,...,1.0,0.0,outdoor_(81).jpg,10.286723,0.931679,12.252411,1.506221,3.537933,1.0,1
2,101,1,nl,0,10_28,0.719317,3,2,1,1,...,1.0,0.0,outdoor_(67).jpg,17.298038,1.295149,25.645838,1.507751,3.026509,0.0,1
3,101,1,nl,0,10_28,0.719317,4,1,3,1,...,6.0,1.0,indoor_(122).jpg,30.180110,1.188015,32.398769,1.502938,2.776548,1.0,1
4,101,1,nl,0,10_28,0.719317,5,1,4,1,...,6.0,0.0,outdoor_(270).jpg,36.678267,1.085774,38.815834,1.505155,2.776437,1.0,1


In [174]:
def choice_positive_outcomes(row):
    ssid,index,lcue,rcue,choice,outcome = (row['SubjNum'], row.name,
                                           row['CueLeftPic'], row['CueRightPic'],
                                           row['CueChosen'], row['Outcome'])
    choice_outcomes = data.take(np.arange(index))[
        data['SubjNum'] == ssid][data['CueChosen'] == choice
    ]['Outcome']
    positive_outcomes = choice_outcomes.sum()
    return(positive_outcomes)

data['choicerewards'] = data.apply(choice_positive_outcomes,axis=1)


C:\Users\ia406477\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [175]:
def choice_total_outcomes(row):
    ssid,index,lcue,rcue,choice,outcome = (row['SubjNum'], row.name,
                                           row['CueLeftPic'], row['CueRightPic'],
                                           row['CueChosen'], row['Outcome'])
    choice_outcomes = data.take(np.arange(index))[
        data['SubjNum'] == ssid][data['CueChosen'] == choice
    ]['Outcome']
    total_outcomes = choice_outcomes.count()
    return(total_outcomes)

data['choicetotal'] = data.apply(choice_total_outcomes,axis=1)

C:\Users\ia406477\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [176]:
data['choicefreq'] = data['choicerewards'] / data['choicetotal']

In [177]:
def altcue_positive_outcomes(row):
    ssid,index,lcue,rcue,choice,outcome = (row['SubjNum'], row.name,
                                           row['CueLeftPic'], row['CueRightPic'],
                                           row['CueChosen'], row['Outcome'])
    altcue = [x for x in [lcue,rcue] if choice != x][0]
    altcue_outcomes = data.take(np.arange(index))[
        data['SubjNum'] == ssid][data['CueChosen'] == altcue
    ]['Outcome']
    positive_outcomes = altcue_outcomes.sum()
    return(positive_outcomes)

data['altcuerewards'] = data.apply(altcue_positive_outcomes,axis=1)

C:\Users\ia406477\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [178]:
def altcue_total_outcomes(row):
    ssid,index,lcue,rcue,choice,outcome = (row['SubjNum'], row.name,
                                           row['CueLeftPic'], row['CueRightPic'],
                                           row['CueChosen'], row['Outcome'])
    altcue = [x for x in [lcue,rcue] if choice != x][0]
    altcue_outcomes = data.take(np.arange(index))[
        data['SubjNum'] == ssid][data['CueChosen'] == altcue
    ]['Outcome']
    total_outcomes = altcue_outcomes.count()
    return(total_outcomes)

data['altcuetotal'] = data.apply(altcue_total_outcomes,axis=1)

C:\Users\ia406477\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [179]:
data['altcuefreq'] = data['altcuerewards'] / data['altcuetotal']

In [180]:
def compare_frequencies(row):
    choicefreq,altfreq = (row['choicefreq'],row['altcuefreq'])
    if choicefreq > altfreq: return 1
    elif choicefreq < altfreq: return 0 
    else: return 99

data['ChoiceOptimal'] = data.apply(compare_frequencies,axis=1)

In [189]:
fpath = os.path.join(outputdir,'memrl_data.csv')
data.to_csv(fpath,index=0)